In [1]:
import pandas as pd
import time
from google.colab import files

# Paso 1: Leer el archivo CSV original
input_csv = '/content/TablaFallas.csv'  # Ruta del archivo original
output_csv_mod = '/content/TablaFallas_modificado.csv'  # Ruta para el archivo modificado
output_csv_no_empty = '/content/TablaFallas_sin_vacios.csv'  # Ruta para el archivo sin vacíos
output_txt_line_protocol = '/content/TablaFallas_line_protocol.txt'  # Ruta para el archivo Line Protocol
output_txt_adjusted = '/content/TablaFallas.txt'  # Ruta para el archivo ajustado con timestamps

# Cargar los datos del CSV original
df = pd.read_csv(input_csv)

# Paso 2: Eliminar columnas no deseadas y renombrar 'Fecha y Hora' a 'Fecha'
df.drop(columns=['Descripcion de Falla', 'Mes', 'Día', 'Semana', 'AÑO'], inplace=True)
df.rename(columns={'Fecha y Hora': 'Fecha'}, inplace=True)

# Guardar el DataFrame modificado
df.to_csv(output_csv_mod, index=False, encoding='utf-8')
print("Paso 2: Columnas eliminadas y columna 'Fecha' renombrada.")

# Paso 3: Reemplazar valores vacíos con 'Desconocido'
df = pd.read_csv(output_csv_mod)
df.fillna("Desconocido", inplace=True)

# Guardar el DataFrame sin vacíos
df.to_csv(output_csv_no_empty, index=False, encoding='utf-8')
print("Paso 3: Campos vacíos reemplazados por 'Desconocido'.")

# Paso 4: Convertir datos a Line Protocol
df = pd.read_csv(output_csv_no_empty)
line_protocol_lines = []

# Función para escapar espacios
def escape_spaces(name):
    return str(name).replace(" ", "\\ ")

# Recorrer cada fila para crear líneas de Line Protocol
for index, row in df.iterrows():
    planta = escape_spaces(row['Planta'])
    area = escape_spaces(row['Area'])
    subarea = escape_spaces(row['Subarea'])
    plc = escape_spaces(row['PLC'])
    grupo = escape_spaces(row['Group'])
    receta = row['Receta']
    objeto = row['Objeto']
    tag = row['Tag']

    timestamp = int(time.mktime(time.strptime(row['Fecha'], "%d.%m.%Y %H:%M:%S"))) * 1_000_000_000

    line = (
        f"fallas,Planta={planta},Area={area},Subarea={subarea},PLC={plc},Group={grupo} "
        f"Tag=\"{tag}\",Objeto=\"{objeto}\",Receta=\"{receta}\" {timestamp}"
    )
    line_protocol_lines.append(line)

# Guardar Line Protocol en un archivo .txt
with open(output_txt_line_protocol, 'w', encoding='utf-8') as f:
    for line in line_protocol_lines:
        f.write(line + "\n")

print("Paso 4: Datos convertidos a Line Protocol y guardados en archivo.")

# Paso 5: Ajustar timestamps repetidos
timestamp_count = {}
adjusted_lines = []

with open(output_txt_line_protocol, 'r', encoding='utf-8') as f:
    lines = f.readlines()

for line in lines:
    parts = line.strip().rsplit(' ', 1)
    if len(parts) != 2:
        continue

    line_data, timestamp_str = parts
    timestamp = int(timestamp_str)

    if timestamp in timestamp_count:
        timestamp_count[timestamp] += 1
        adjusted_timestamp = timestamp + timestamp_count[timestamp]
    else:
        timestamp_count[timestamp] = 0
        adjusted_timestamp = timestamp

    adjusted_line = f"{line_data} {adjusted_timestamp}"
    adjusted_lines.append(adjusted_line)

# Guardar las líneas ajustadas
with open(output_txt_adjusted, 'w', encoding='utf-8') as f:
    for adjusted_line in adjusted_lines:
        f.write(adjusted_line + "\n")

print("Paso 5: Timestamps ajustados y guardados en archivo.")

# Paso 6: Descargar el archivo ajustado
files.download(output_txt_adjusted)
print("Paso 6: Archivo descargado.")

Paso 2: Columnas eliminadas y columna 'Fecha' renombrada.
Paso 3: Campos vacíos reemplazados por 'Desconocido'.
Paso 4: Datos convertidos a Line Protocol y guardados en archivo.
Paso 5: Timestamps ajustados y guardados en archivo.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Paso 6: Archivo descargado.
